Выполнить Стэкинг, Бэгинг, Вотинг и Бустинг. При реализации алгоритмов не использовать готовые решения. 
За сравнение взять CatBoostClassifier как базовая метрика качества. Сравнить результат с реализацией своих ансамблей. 
Для однозначности и интерпретируемости результатов использовать приложенный набор данных. 
При реализации бустинга - просто сокращайте набор данных на котором модель отработала хорошо (правильно предсказанные данные). 

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, accuracy_score, f1_score
from catboost import CatBoostClassifier, Pool
from sklearn.base import clone

In [3]:
# Загрузка датасета
data = pd.read_csv('winequality-white.csv', delimiter=';')

In [4]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


Посмотрим на соотношение классов в датасете.

In [5]:
# Считаем количество каждого класса
class_counts = data['quality'].value_counts().sort_index()

# Считаем процентное соотношение
class_percentages = (class_counts / class_counts.sum()) * 100

# Вывод результатов
print("Процентное соотношение классов (quality):")
for quality, percentage in class_percentages.items():
    print(f"Класс {quality}: {percentage:.2f}%")

Процентное соотношение классов (quality):
Класс 3: 0.41%
Класс 4: 3.33%
Класс 5: 29.75%
Класс 6: 44.88%
Класс 7: 17.97%
Класс 8: 3.57%
Класс 9: 0.10%


Классы несбалансированы. Применим RandomOverSampler для балансировки.

In [6]:
y = data['quality']
X = data.drop('quality', axis=1)

In [8]:
# 1. Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
# Применяем RandomOverSampler к тренировочным данным
ros = RandomOverSampler(random_state=42)
X_train, y_train = ros.fit_resample(X_train, y_train)

In [10]:
# Масштабируем признаки
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Обучим и проверим CatBoostClassifier

In [11]:
# Создаем объект Pool
train_pool = Pool(X_train_scaled, y_train)
test_pool = Pool(X_test_scaled, y_test)

In [12]:
space = {
    "iterations": hp.choice("iterations", range(100, 2000)),  # Количество деревьев
    "depth": hp.choice("depth", range(3, 10)),  # Глубина деревьев
    "learning_rate": hp.loguniform("learning_rate", -3, 0),  # Темп обучения (0.001 - 1)
    "l2_leaf_reg": hp.uniform("l2_leaf_reg", 1, 10),  # Регуляризация L2
    "border_count": hp.choice("border_count", range(32, 255)),  # Количество бинов разбиения
    "bagging_temperature": hp.uniform("bagging_temperature", 0, 1),  # Температура бэггинга
}

In [37]:
def objective(params):
    model = CatBoostClassifier(
        iterations=int(params["iterations"]),
        depth=int(params["depth"]),
        learning_rate=params["learning_rate"],
        l2_leaf_reg=params["l2_leaf_reg"],
        border_count=int(params["border_count"]),
        bagging_temperature=params["bagging_temperature"],
        verbose=0
    )

    model.fit(X_train_scaled, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=0)
    
    y_pred = model.predict(X_test_scaled)
    accuracy = f1_score(y_test, y_pred, average = 'weighted')

    return {"loss": -accuracy, "status": STATUS_OK} 

In [38]:

trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,  # Количество итераций оптимизации
    trials=trials
)

print("Лучшие параметры:", best_params)

100%|██████████| 50/50 [03:21<00:00,  4.03s/trial, best loss: -0.6534166160595409]
Лучшие параметры: {'bagging_temperature': 0.6204685008777656, 'border_count': 103, 'depth': 6, 'iterations': 1763, 'l2_leaf_reg': 1.9900607052193315, 'learning_rate': 0.15275517180137332}


In [39]:
%%capture
model = CatBoostClassifier(**best_params)
model.fit(train_pool, eval_set=test_pool)

In [43]:
# 8. Оценка модели на тестовой выборке
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"accuracy: {accuracy}")

accuracy: 0.6275510204081632


Метрика accuracy модели CatBoostClassifier - 0.62

Теперь реализуем bagging, voting, stacking.

In [17]:

def bagging(X_train, y_train, X_test, models, n_estimators=10, sample_fraction=0.8, random_state=None):
    """Бэггинг: усредняет предсказания нескольких моделей, обученных на случайных подвыборках."""
    n_samples = int(len(X_train) * sample_fraction)
    predictions = np.zeros((n_estimators, len(X_test)), dtype=int)

    # Фиксируем случайность, если указан random_state
    if random_state is not None:
        np.random.seed(random_state)

    for i in range(n_estimators):
        idxs = np.random.choice(len(X_train), n_samples, replace=True) # Бутстрэп
        X_sample, y_sample = X_train[idxs], y_train.iloc[idxs]
        
        model = clone(models[i % len(models)])  # Берем модель из списка
        model.fit(X_sample, y_sample)
        
        predictions[i] = model.predict(X_test)

    # Мажоритарное голосование
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)
    
    return final_predictions



In [19]:
def voting(X_train, y_train, X_test, models, voting_type="hard"):
    """Вотинг: усредняет предсказания моделей (hard – по голосам, soft – по вероятностям)."""
    trained_models = [clone(model).fit(X_train, y_train) for model in models]
    predictions = np.array([model.predict(X_test) for model in trained_models])

    if voting_type == "hard":
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)
    else:
        probs = np.array([model.predict_proba(X_test) for model in trained_models])
        avg_probs = np.mean(probs, axis=0)
        return np.argmax(avg_probs, axis=1)

In [20]:
from sklearn.base import clone
import numpy as np

def stacking(X_train, y_train, X_test, models, meta_model):
    """Стекинг: обучаем модели, собираем предсказания и обучаем мета-модель"""
    
    trained_models = []
    
    # Обучаем базовые модели и собираем предсказания для мета-признаков
    meta_features_train = np.column_stack([
        clone(model).fit(X_train, y_train).predict(X_train) for model in models
    ])
    
    # Теперь обучаем сами модели (они понадобятся для теста)
    for model in models:
        model.fit(X_train, y_train)
        trained_models.append(model)
    
    # Предсказываем на тесте
    meta_features_test = np.column_stack([model.predict(X_test) for model in trained_models])

    # Обучаем мета-модель
    meta_model.fit(meta_features_train, y_train)
    
    return meta_model.predict(meta_features_test)


Будем проверять методы на моделях DecisionTreeClassifier, RandomForestClassifier, LogisticRegression (как результирующая модель для Стекинга)

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Посмотрим метрики моделей по отдельности

In [22]:
#DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.5827


In [23]:
#RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.6571


In [24]:
#LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.1939


d:\soft\Anaconda\envs\ML1\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


И проверим собственные реализации вотинга, бэггинга и стекинга

In [25]:
# Задаем модели
models = [DecisionTreeClassifier(), RandomForestClassifier()]
meta_model = LogisticRegression()

# Вызываем методы
y_pred_bagging = bagging(X_train_scaled, y_train, X_test_scaled, models)
y_pred_stacking = stacking(X_train_scaled, y_train, X_test_scaled, models, meta_model)
y_pred_voting = voting(X_train_scaled, y_train, X_test_scaled, models, voting_type="hard")

# Выводим точности
print("Bagging Accuracy:", accuracy_score(y_test, y_pred_bagging))
print("Stacking Accuracy:", accuracy_score(y_test, y_pred_stacking))
print("Voting Accuracy:", accuracy_score(y_test, y_pred_voting))

d:\soft\Anaconda\envs\ML1\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Bagging Accuracy: 0.6448979591836734
Stacking Accuracy: 0.6173469387755102
Voting Accuracy: 0.639795918367347


Реализуем Бустинг на основе деревьев решений

In [26]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

def boosting(X_train, y_train, X_test, n_estimators=100, learning_rate=0.1, max_depth=3):
   
    # Инициализация нулевого предсказания (среднее значение таргета)
    predictions = np.full_like(y_train, np.mean(y_train), dtype=np.float64)
    
    models = []  # Список моделей

    for _ in range(n_estimators):
        # Вычисляем градиент (ошибку между предсказаниями и реальными значениями)
        residuals = y_train - predictions

        # Обучаем слабый базовый алгоритм (решающее дерево)
        model = DecisionTreeRegressor(max_depth=max_depth)
        model.fit(X_train, residuals)

        # Добавляем слабую модель в список
        models.append(model)

        # Обновляем предсказания, добавляя скорректированные ошибки с учетом learning_rate
        predictions += learning_rate * model.predict(X_train)

    # Финальное предсказание для тестовой выборки
    final_predictions = np.full_like(y_test, np.mean(y_train), dtype=np.float64)
    for model in models:
        final_predictions += learning_rate * model.predict(X_test)

    return final_predictions.round()


In [27]:

y_pred_boosting = boosting(X_train_scaled, y_train, X_test_scaled,n_estimators=5000, learning_rate=0.2)


# Выводим точности
print("Bagging Accuracy:", accuracy_score(y_test, y_pred_boosting))


Bagging Accuracy: 0.6336734693877552


Эталонное значение accuracy модели CatBoost - 0.931, подбор параметров выполнен с помощью Hyperopt.
При применении ансамблей лучший результат получен при использовании Voting - 0.933, однако он меньше результата CatBoost и меньше чем accuracy при использовании чистого RandomForestClassifier.
В моем случае, применение ансамблей усредняет результаты моделей (для Voting и Bagging). При использовании Stacking слабая модель LogisticRegression не сильно влияет на итоговый результат.

Реализация бустинга на основе деревьев решений дала результат accuracy 0.928. Результат напрямую зависит от количества базовых моделей (рещающих деревьев).